# Carga de Datos

##### Futuras mejoras
* Agregar la diferencia de tiempo en que se consiguió controlar el incendio
* Analizar mejor de qué forma se distribuyen los datos nulos y de qué forma puedo rellenar los valores faltantes, si es posible.

## Importación de librerías

In [9]:
from os import path
import pandas as pd
import sqlite3
import datetime as dt

## Constantes

Rutas usuales que se ocuparán en el notebook

In [10]:
# Salvar DataFrame?
SAVE_DF = True

# RUTAS
MAIN_PATH = path.join("..")

DATA_PATH = path.join(MAIN_PATH, "data")
WF_FOLDER_PATH = path.join(DATA_PATH, "wildfires_us")
WF_DATA_PATH = path.join(WF_FOLDER_PATH, "FPA_FOD_20170508.sqlite")
WF_DATA_COLUMNS_PATH = path.join(WF_FOLDER_PATH, "COLUMNS")

## Funciones auxiliares

Función que se ocupará en la carga de datos.

In [11]:
def crear_puntero(path=WF_DATA_PATH, msg=False):
    """Función que crea un puntero con el dataset de incendios forestales en US.
    """
    try:
        conn = sqlite3.connect(path)
        if msg:
            print("Conexión realizada con éxito.")
    except:
        if msg:
            print("No se encuentra el archivo.")
    return conn

Función que se ocupará para imprimir la información (número de filas y columnas) de un DataFrame.

In [12]:
def print_cantidad(dataframe):
    """Imprime la cantidad de datos que tiene el Data Frame.
    """
    msg_cantidad = "El dataset tiene una cantidad de {} datos y {} variables."
    print(msg_cantidad.format(dataframe.shape[0], dataframe.shape[1]))
    return None

## Carga de datos

### Columnas a ocupar

Se escojen las columnas a ocupar dependiendo de la importancia que tenga. Se omiten algunas columnas tales como las que son para el ID, como el nombre que tuvo el incendio, o la columna que indica de dónde se obtuvo el incendio; pues no deberían de afectar a la predicción.

In [13]:
# Todas las columnas
columnas = list(pd.read_csv(WF_DATA_COLUMNS_PATH))

# Columnas que se ocuparán en el análisis
columnas_ocupadas = [
    "FIRE_YEAR",        # Año en que el incendio fue descubierto o confirmado
    "DISCOVERY_DATE",   # Fecha en que fue descubiertoel incendio
    "DISCOVERY_TIME",   # Hora en que fue descubierto el incendio
    "STAT_CAUSE_DESCR", # Descripción de la causa del incendio
    "CONT_DATE",        # Fecha en que se contuvo el incendio
    "CONT_TIME",        # Hora en que se contuvo el incendio
    "FIRE_SIZE",        # Área final estimada que alcanzó el incendio
    "FIRE_SIZE_CLASS",
    "LATITUDE",         # Latitud de la localización del incendio
    "LONGITUDE",        # Longitud de la localización del incendio
    "STATE",
]

# Columnas que no se ocuparán
columnas_sin_ocupar = [x for x in columnas if x not in columnas_ocupadas]

### Carga de Datos

In [16]:
# str que se ocupará al cargar los datos
columnas_str = ",".join(columnas_ocupadas)

# Obtener una fila del dataset
conn = crear_puntero(msg=True)
df = pd.read_sql_query(f"SELECT {columnas_str} FROM 'Fires'", conn)
conn.close()

del columnas_str, conn

print_cantidad(df)

# df = df[df["FIRE_YEAR"].map(int) >= 2011].reset_index()

print_cantidad(df)

df.head()

Conexión realizada con éxito.
El dataset tiene una cantidad de 1880465 datos y 11 variables.
El dataset tiene una cantidad de 1880465 datos y 11 variables.


,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_TIME,STAT_CAUSE_DESCR,CONT_DATE,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE
0,2005,2453403.5,1300,Miscellaneous,2453403.5,1730,0.10,A,40.036944,-121.005833,CA
1,2004,2453137.5,0845,Lightning,2453137.5,1530,0.25,A,38.933056,-120.404444,CA
2,2004,2453156.5,1921,Debris Burning,2453156.5,2024,0.10,A,38.984167,-120.735556,CA
3,2004,2453184.5,1600,Lightning,2453189.5,1400,0.10,A,38.559167,-119.913333,CA
4,2004,2453184.5,1600,Lightning,2453189.5,1200,0.10,A,38.559167,-119.933056,CA


## Descripción preliminar de los datos

Observemos la información actual del df

In [8]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   FIRE_YEAR         int64  
 1   DISCOVERY_DATE    float64
 2   DISCOVERY_TIME    object 
 3   STAT_CAUSE_DESCR  object 
 4   CONT_DATE         float64
 5   CONT_TIME         object 
 6   FIRE_SIZE         float64
 7   FIRE_SIZE_CLASS   object 
 8   LATITUDE          float64
 9   LONGITUDE         float64
 10  STATE             object 
dtypes: float64(5), int64(1), object(5)
memory usage: 157.8+ MB


Observamos que 6 variables son del tipo numérico, y 5 son strings. Sin embargo, `DISCOVERY_DATE` y `CONT_DATE` son fechas. Esto se tratará más adelante.

### Valores duplicados

Revisamos si no tiene datos duplicados

In [9]:
n_dup = sum(df.duplicated())
print("Existen", n_dup, "valores duplicados")
del n_dup

Existen 3535 valores duplicados


Dado que existen valores duplicados en el dataset, los dropeamos.

In [10]:
df.drop_duplicates(inplace=True, ignore_index=True)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1876930 entries, 0 to 1876929
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   FIRE_YEAR         int64  
 1   DISCOVERY_DATE    float64
 2   DISCOVERY_TIME    object 
 3   STAT_CAUSE_DESCR  object 
 4   CONT_DATE         float64
 5   CONT_TIME         object 
 6   FIRE_SIZE         float64
 7   FIRE_SIZE_CLASS   object 
 8   LATITUDE          float64
 9   LONGITUDE         float64
 10  STATE             object 
dtypes: float64(5), int64(1), object(5)
memory usage: 157.5+ MB


### Manejo de valores nulos

Primero observamos el número de elementos nulos que tiene el dataset.

In [17]:
df.isna().sum()

FIRE_YEAR                0
DISCOVERY_DATE           0
DISCOVERY_TIME      882638
STAT_CAUSE_DESCR         0
CONT_DATE           891531
CONT_TIME           972173
FIRE_SIZE                0
FIRE_SIZE_CLASS          0
LATITUDE                 0
LONGITUDE                0
STATE                    0
dtype: int64

Dado al gran tamaño de la base de datos, nos podemos dar "el lujo" de eliminar las filas que contengan elementos nulos.

In [12]:
# Se quitan los valores nulos
df.dropna(inplace=True)

# Se resetean los índices
df.reset_index(drop=True, inplace=True)

# Se imprime la cantidad
print_cantidad(df)

# Y se muestran la cantidad de nulos que tiene actualmente
df.isna().sum()

El dataset tiene una cantidad de 890821 datos y 11 variables.


FIRE_YEAR           0
DISCOVERY_DATE      0
DISCOVERY_TIME      0
STAT_CAUSE_DESCR    0
CONT_DATE           0
CONT_TIME           0
FIRE_SIZE           0
FIRE_SIZE_CLASS     0
LATITUDE            0
LONGITUDE           0
STATE               0
dtype: int64

Ahora revisamos el tipo de datos que tiene el Data Frame

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890821 entries, 0 to 890820
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   FIRE_YEAR         890821 non-null  int64  
 1   DISCOVERY_DATE    890821 non-null  float64
 2   DISCOVERY_TIME    890821 non-null  object 
 3   STAT_CAUSE_DESCR  890821 non-null  object 
 4   CONT_DATE         890821 non-null  float64
 5   CONT_TIME         890821 non-null  object 
 6   FIRE_SIZE         890821 non-null  float64
 7   FIRE_SIZE_CLASS   890821 non-null  object 
 8   LATITUDE          890821 non-null  float64
 9   LONGITUDE         890821 non-null  float64
 10  STATE             890821 non-null  object 
dtypes: float64(5), int64(1), object(5)
memory usage: 74.8+ MB


Terminamos con un dataset con la mitad de datos de cómo fue originalmente.

### Manejo de las variables de tiempo

#### Creación de la columnas `{}_DATE_TIME`, a partir de las dos anteriores

Notemos que `{DISCOVERY,CONT}_DATE` está en formato timestamp, y que `{DISCOVERY,CONT}_TIME` está en formato `'hhmm'`:

In [14]:
cols_aux = ["DISCOVERY_DATE", "DISCOVERY_TIME", 
            "CONT_DATE", "CONT_TIME"]

df[cols_aux].head()

,DISCOVERY_DATE,DISCOVERY_TIME,CONT_DATE,CONT_TIME
0,2453403.5,1300,2453403.5,1730
1,2453137.5,0845,2453137.5,1530
2,2453156.5,1921,2453156.5,2024
3,2453184.5,1600,2453189.5,1400
4,2453184.5,1600,2453189.5,1200


Generamos una nueva columna que tenga la fecha y hora contenida. Pues la fecha está en formato Timestamp y la hora está en formato `'hhmm'`. Lo hacemos para los prefijos `'DISCOVERY_'` y `'CONT_'`.

In [15]:
def generar_datetime(fecha_col, hora_col):
    """Recibe un DataFrame 'df', una columna 'fecha_col' que tiene 
    la fecha en formato de marca de tiempo (Timestamp) y una columna 
    'hora_col' que tiene la hora en formato 'hhmm'.

    Retorna una columna del tipo 'Series', de datos del tipo datetime,
    con la fecha y hora juntas.
    """

    # Transformamos la columna de fechas a la fecha "usual"
    fecha_col_ = pd.to_datetime(fecha_col - pd.Timestamp(0).to_julian_date(),
                                unit='D',
                                dayfirst=True)
    
    # Obtenemos solo la parte de la fecha como una columna de str
    fecha_col_ = fecha_col_.map(lambda x: str(x).split()[0])

    # Procesamos la columna de la hora para que esté en formato "hh:mm:ss"
    hora_col_ = hora_col.map(lambda x: ":".join([" " + x[0:2], x[2:4], "00"]))

    # Retornamos la suma de ambas, en formato de datetime
    return pd.to_datetime(fecha_col_ + hora_col_)

In [16]:
# Creamos nuevas columnas
df["DISC_DATE_TIME"] = generar_datetime(df["DISCOVERY_DATE"],
                                        df["DISCOVERY_TIME"])
df["CONT_DATE_TIME"] = generar_datetime(df["CONT_DATE"],
                                        df["CONT_TIME"])

# Agregamos estas nuevas columnas al conjunto de columnas
nuevas_cols = ["DISC_DATE_TIME", "CONT_DATE_TIME"]
columnas += nuevas_cols
columnas_ocupadas += nuevas_cols

df[nuevas_cols].head()

,DISC_DATE_TIME,CONT_DATE_TIME
0,2005-02-02 13:00:00,2005-02-02 17:30:00
1,2004-05-12 08:45:00,2004-05-12 15:30:00
2,2004-05-31 19:21:00,2004-05-31 20:24:00
3,2004-06-28 16:00:00,2004-07-03 14:00:00
4,2004-06-28 16:00:00,2004-07-03 12:00:00


Quitamos ahora las columnas que resultan redundantes, al tener la misma información de los que tiene prefijo `_DATE_TIME`: 
* `DISCOVERY_DATE`, `DISCOVERY_TIME`.
* `CONT_DATE`, `CONT_TIME`.

In [17]:
for x in cols_aux:
    columnas_ocupadas.remove(x)
    columnas_sin_ocupar.append(x)

del cols_aux, nuevas_cols
    
df = df[columnas_ocupadas]
print_cantidad(df)
df.head()

El dataset tiene una cantidad de 890821 datos y 9 variables.


,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,CONT_DATE_TIME
0,2005,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02 13:00:00,2005-02-02 17:30:00
1,2004,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12 08:45:00,2004-05-12 15:30:00
2,2004,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31 19:21:00,2004-05-31 20:24:00
3,2004,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28 16:00:00,2004-07-03 14:00:00
4,2004,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28 16:00:00,2004-07-03 12:00:00


#### Creación de las columnas `{}_MONTH` y `{}_DAY_OF_WEEK`, a partir de `{}_DATE_TIME`

Crearemos una columna de los meses y semanas, para ver si nos entrega información extra.

In [18]:
def number_to_month(n):
    """Hace un mapeo del número de un mes, al nombre del mes.
    Retorna None si no se entrega un número entero entre el 1 y el 12.
    """
    
    m = {
        1:"January", 2:"February", 3:"March", 4:"April",
        5:"May", 6:"June", 7:"July", 8:"August",
        9:"September", 10:"Octuber", 11:"November", 12:"December"
    }
    
    return m.get(n, None)

In [19]:
# Creamos las columnas {DISC,CONT}_{MONTH,DAY_OF_WEEK}
for x in ["DISC", "CONT"]:
    df[f"{x}_MONTH"] = df[f"{x}_DATE_TIME"].dt.month.map(number_to_month)
    df[f"{x}_DOW"] = df[f"{x}_DATE_TIME"].dt.day_name()
    df[f"{x}_TIME"] = df[f"{x}_DATE_TIME"].dt.hour.map(int)

# Añadimos las nuevas columnas al conjunto de columnas
cdt = columnas_ocupadas.pop(-1)
columnas.pop(-1)
cols_aux = ["DISC_MONTH", "DISC_DOW", "DISC_TIME",
            cdt, "CONT_MONTH", "CONT_DOW", "CONT_TIME"]

columnas.extend(cols_aux)
columnas_ocupadas.extend(cols_aux)
del cdt, cols_aux

# Actualizamos el df
df = df[columnas_ocupadas]

df.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME
0,2005,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02 13:00:00,February,Wednesday,13,2005-02-02 17:30:00,February,Wednesday,17
1,2004,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12 08:45:00,May,Wednesday,8,2004-05-12 15:30:00,May,Wednesday,15
2,2004,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31 19:21:00,May,Monday,19,2004-05-31 20:24:00,May,Monday,20
3,2004,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28 16:00:00,June,Monday,16,2004-07-03 14:00:00,July,Saturday,14
4,2004,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28 16:00:00,June,Monday,16,2004-07-03 12:00:00,July,Saturday,12


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890821 entries, 0 to 890820
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   FIRE_YEAR         890821 non-null  int64         
 1   STAT_CAUSE_DESCR  890821 non-null  object        
 2   FIRE_SIZE         890821 non-null  float64       
 3   FIRE_SIZE_CLASS   890821 non-null  object        
 4   LATITUDE          890821 non-null  float64       
 5   LONGITUDE         890821 non-null  float64       
 6   STATE             890821 non-null  object        
 7   DISC_DATE_TIME    890821 non-null  datetime64[ns]
 8   DISC_MONTH        890821 non-null  object        
 9   DISC_DOW          890821 non-null  object        
 10  DISC_TIME         890821 non-null  int64         
 11  CONT_DATE_TIME    890821 non-null  datetime64[ns]
 12  CONT_MONTH        890821 non-null  object        
 13  CONT_DOW          890821 non-null  object        
 14  CONT

#### Obtener las diferencias de tiempo del control del incendio

In [21]:
df["DT_FIRE"] = df["CONT_DATE_TIME"] - df["DISC_DATE_TIME"]
df.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,2005,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02 13:00:00,February,Wednesday,13,2005-02-02 17:30:00,February,Wednesday,17,0 days 04:30:00
1,2004,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12 08:45:00,May,Wednesday,8,2004-05-12 15:30:00,May,Wednesday,15,0 days 06:45:00
2,2004,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31 19:21:00,May,Monday,19,2004-05-31 20:24:00,May,Monday,20,0 days 01:03:00
3,2004,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28 16:00:00,June,Monday,16,2004-07-03 14:00:00,July,Saturday,14,4 days 22:00:00
4,2004,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28 16:00:00,June,Monday,16,2004-07-03 12:00:00,July,Saturday,12,4 days 20:00:00


In [22]:
df["DT_FIRE"] = df.DT_FIRE.map(lambda x: x.total_seconds()/3600)
df.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,2005,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02 13:00:00,February,Wednesday,13,2005-02-02 17:30:00,February,Wednesday,17,4.50
1,2004,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12 08:45:00,May,Wednesday,8,2004-05-12 15:30:00,May,Wednesday,15,6.75
2,2004,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31 19:21:00,May,Monday,19,2004-05-31 20:24:00,May,Monday,20,1.05
3,2004,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28 16:00:00,June,Monday,16,2004-07-03 14:00:00,July,Saturday,14,118.00
4,2004,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28 16:00:00,June,Monday,16,2004-07-03 12:00:00,July,Saturday,12,116.00


## Orden por fecha de descubrimiento

In [23]:
df.sort_values(by=["DISC_DATE_TIME"], ignore_index=True, inplace=True)

In [24]:
df.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,1992,Miscellaneous,0.36,B,32.4410,-82.7018,GA,1992-01-01 00:01:00,January,Wednesday,0,1992-01-01 01:14:00,January,Wednesday,1,1.216667
1,1992,Miscellaneous,1.02,B,32.2009,-82.2967,GA,1992-01-01 00:05:00,January,Wednesday,0,1992-01-01 00:40:00,January,Wednesday,0,0.583333
2,1992,Miscellaneous,0.58,B,32.1325,-82.7610,GA,1992-01-01 00:10:00,January,Wednesday,0,1992-01-01 01:15:00,January,Wednesday,1,1.083333
3,1992,Children,10.14,C,32.0300,-83.9350,GA,1992-01-01 00:30:00,January,Wednesday,0,1992-01-01 02:20:00,January,Wednesday,2,1.833333
4,1992,Lightning,0.10,A,38.2050,-120.3350,CA,1992-01-01 01:30:00,January,Wednesday,1,1992-01-01 02:10:00,January,Wednesday,2,0.666667


## Salvar DataFrame

Salvamos el DataFrame en la carpeta respectiva.

In [26]:
if SAVE_DF:
    df.to_csv(path.join(WF_FOLDER_PATH, "WILDFIRES_USA.csv"), index=False)
    print("Dataset guardado.")
    s = ",".join(list(df.columns))
    pd.DataFrame(data={s:[0]}).to_csv(path.join(WF_FOLDER_PATH, 
                                                "WILDFIRES_USA_COLUMNS.csv"), 
                                      index=False)
    print("Columnas del dataset guardadas.")
    del s

Dataset guardado.
Columnas del dataset guardadas.
